# Übung 1 - Modellierung von PV-Stromerzeugung

Gruppe 2/B

Autoren: Ahmed Akhi, Johannes Gerger, Markus Heitzinger

In [5]:
# Import von Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

__Vorgegebene Funktion zur Bestimmung des relativen Sonnenstands:__

In [6]:
def sonnenstand(laengengrad,breitengrad,time):
    """
    Diese Funktion erstellt einen Vektor für den Sonnenstand in 15 min
    Auflösung über ein Jahr

    die Variable "azimut" ergibt einen Vektor des azimuts über ein Jahr
    der azimut ergibt nur für positive Hoehenwinkel verlässliche Werte, da
    die atan-Funktion nur Abweichungen bis zu 90° liefert. In weiteren 
    Berechnungen sollten also nur jene Werte mit positiven Hoehenwinkeln 
    (nach Sonnenaufgang) verwendet werden. Für die Berechnung von PV-Erträgen
    reicht dies vollkommen aus. 
    die Variable "hoehenwinkel" ergibt einen Vektor aller hoehenwinkel über 
    ein Jahr
    für Schaltjahre muss die Berechnung dementsprechend angepasst werden. Dies
    Anpassung muss im Rahmen dieser LV allerdings nicht berücksichtigt werden.

    Quellen zur Berechnung des Sonnenstands: Ursula Eicker(2012), 
    Jakob Anger(2012), Rainer Blabensteiner(2011)

    Args:
        laengengrad: float
        breitengrad: float
        time: pandas.Dataframe

    Returns:
        azimut: numpy.array
        hoehenwinkel: numpy.array
    
    """
    ## Zeitgleichung zur Berechnung der wahren Ortszeit
    
    # Zeitgleichung - ergibt die 
    # Abweichung der Sonnenuhr von der mittlerer Ortszeit(MOZ)
    hw1 = 360 / 365 * time['Tag'] # Hilfswinkel 1
    
    z = 0.008 * np.cos(np.radians(hw1)) - 0.122 * np.sin(np.radians(hw1)) \
        - 0.052 * np.cos(np.radians(2 * hw1)) - 0.157 * np.sin(np.radians(2 * hw1)) \
        - 0.001 * np.cos(np.radians(3 * hw1)) - 0.005 * np.sin(np.radians(3 * hw1))

    WOZ = time['Stunden'] + z + 1 / 15 * laengengrad - math.floor(1/15 * laengengrad) # Wahre Ortszeit (WOZ), der Vektor 
    # time.Stunden entspricht der mittleren Ortszeit (MOZ) in Stunden
    
    ## Stundenwinkel bei Berechnung über True Solar Time
    # time.Stunden entspricht hier der Sonnenzeit und nicht der Ortszeit
    # Stw=15*time.Stunden; #Stundenwinkel 
    # In einer Stunde legt die Sonne 15Grad zurück deswegen *15
    Stw = 15 * WOZ # Stundenwinkel
    
    ## Berechnung der Deklination
    
    Ew = 0.98630 * (np.array(time.Tag) - 2.8749) + 1.9137 * np.sin(np.radians(0.98630 * (np.array(time.Tag) - 2.8749))) + 102.06 # Ekliptikale Länge in Grad    
    Dw = np.degrees(np.arcsin(-0.3979 * np.sin(np.radians(Ew)))) # Winkel zwischen Äquatorebene und Verbindungs-
    # linie Sonne-Erde. Dieser Winkel wird als Sonnendeklination bezeichnet
    # und schwankt über das Jahr zwischen 23°26,5' und -23°26,5'

    ## Berechnung des Höhenwinkels

    hoehenwinkel = np.degrees(np.arcsin(np.sin(np.radians(Dw)) * np.sin(np.radians(breitengrad)) - np.cos(np.radians(Dw)) * np.cos(np.radians(breitengrad)) * np.cos(np.radians(Stw))))

    ## Berechnung des Azimuts
    # Winkel nach Sonnenuntergang werden hier nicht vollständig abgebildet

    azimutfull = 180 + np.degrees(np.arctan(-np.cos(np.radians(Dw)) * np.sin(np.radians(Stw)) / (-np.cos(np.radians(Dw)) * np.sin(np.radians(breitengrad)) * np.cos(np.radians(Stw)) - np.sin(np.radians(Dw)) * np.cos(np.radians(breitengrad)))))
    
    
    azimutfull[hoehenwinkel < -20] = 0 # Eliminiert
    # unter -20°

    azimutaufteilung = np.reshape(np.array(azimutfull), (96, 365), order="F") # teilt Vektor auf Tage auf
    az1 = azimutaufteilung[0:int((len(time['Stunden'])//365)//2), :] # ergibt Werte für die erste Tageshälfte
    az1[az1 > 220] = az1[az1 > 220] - 180 # Korrektur der Winkel für Abweichungen von über 90° aus Südrichtung
    az11 = az1[0:int(len(time['Stunden'])//365*0.25), :]
    az11[az11 >= 180] = az11[az11 >= 180] - 180
    az1[0:len(az11[:, 0]), :] = az11

    az2 = azimutaufteilung[int((len(time['Stunden'])//365)//2):, :] # ergibt Werte für die zweite Tageshälfte
    az2[(az2 < 150) & (az2 > 0)] = az2[(az2 < 150) & (az2 > 0)] + 180 # Korrektur der Winkel für Abweichungen von über 90° aus Südrichtung
    az22 = az2[int(len(az2[:, 0])*0.5):, :]
    az22[(az22 <= 180) & (az22 > 0)] = az22[(az22 <= 180) & (az22 > 0)] + 180
    az2[int(len(az2[:, 0])-len(az22[:, 0])):, :] = az22
    
    print(len(az1) + len(az2))
    azimutaufteilung = np.concatenate((az1, az2), axis=0) # Zusammenfassung der ersten und zweiten Tageshälfte
    azimut = np.reshape(azimutaufteilung, (35040, ), order="F") # reshape der Tage in einen Vektor aller Winkel über den Jahresverlauf in 15min Intervalle

    return azimut, hoehenwinkel


## 1. Modellimplementierung

In [7]:
def jahreserzeugung():


    # !!! TODO
    return None

In [8]:
def jahreserzeugung_temp():
    # !!! TODO
    return None

## 2. Anwendung der Modelle

In [9]:
# Input - Parameter
# !! Achtung: Anpassen an gruppen-spezifische Parameter (siehe Übungsangabe)
laengengrad=16.3
breitengrad=48.2
Installpv = 1
Modulwirkungsgrad = 0.17 # Wirkungsgrad der Panele
SonstigeVerluste = 0.8 # Wirkungsgrad aller anderen Komponenten...
Installpv = 1 # installierte Leistung der Anlage in kWp

# Ausrichtung der PV
pv_hoehenwinkel=20 
pv_azimut=180

Systemkostenkwp=2000 # €/kWp Systemkosten
A = 0.2 #Albedo
kappa = 1.041
E_0 = 1367 #W/m^2

dfTime = pd.read_csv("time.csv");
dfStrahlung = pd.read_csv("Strahlung.csv");
dfTemperatur = pd.read_csv("Temperatur.csv");

dfSonnenstand = pd.DataFrame(sonnenstand(laengengrad, breitengrad,dfTime))
dfSonnenstand = dfSonnenstand.transpose()
dfSonnenstand.rename(columns = {0:'Azimut'}, inplace = True)
dfSonnenstand.rename(columns = {1:'Höhenwinkel'}, inplace = True)

dfSonnenstand[40:50]

96


,Azimut,Höhenwinkel
40,155.550739,15.096052
41,158.989520,16.061875
42,162.483320,16.886545
43,166.025221,17.564877
44,169.607025,18.092435
45,173.219420,18.465658
46,176.852206,18.681965
47,180.494579,18.739838
48,184.135440,18.638865
49,187.763740,18.379762


In [10]:

dfSonnenstand["Moduleinfallswinkel"] = np.arccos(-np.cos(np.radians(dfSonnenstand["Höhenwinkel"]))*np.sin(np.radians(pv_hoehenwinkel)*np.cos(np.radians(dfSonnenstand["Azimut"]-pv_azimut-180)+np.sin(np.radians(dfSonnenstand["Höhenwinkel"])*np.cos(pv_hoehenwinkel)))))
dfStrahlung[0:1]

,Year,Month,Day,Hour,DirectInclined,DiffuseInclined,Reflected,GlobalInclined,DirectHoriz,DiffusHoriz,GlobalHoriz,TopOfAtmosphere
0,2005,1,1,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Create a function to map epsilon to the specified column values
def map_epsilon_to_column(epsilon, column_name):
    if column_name == 'F11':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [-0.008, 0.13, 0.33, 0.568, 0.873, 1.132, 1.06, 0.678]
    elif column_name == 'F12':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [0.588, 0.683, 0.0487, 0.187, -0.392, -1.237, -1.6, -0.327]
    elif column_name == 'F13':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [-0.062, -0.151, -0.221, -0.295, -0.362, -0.412, -0.359, -0.25]
    elif column_name == 'F21':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [-0.06, -0.019, 0.055, 0.109, 0.226, 0.288, 0.264, 0.156]
    elif column_name == 'F22':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [0.072, 0.066, -0.064, -0.152, -0.462, -0.823, -1.127, -1.377]
    elif column_name == 'F23':
        conditions = [
            (epsilon >= 1) & (epsilon < 1.065),
            (epsilon >= 1.065) & (epsilon < 1.23),
            (epsilon >= 1.23) & (epsilon < 1.5),
            (epsilon >= 1.5) & (epsilon < 1.95),
            (epsilon >= 1.95) & (epsilon < 2.8),
            (epsilon >= 2.8) & (epsilon < 4.5),
            (epsilon >= 4.5) & (epsilon < 6.2),
            (epsilon >= 6.2)
        ]
        choices = [-0.022, -0.029, -0.026, -0.014, 0.001, 0.056, 0.131, 0.251]
    else:
        return np.nan  # Default value if column_name is not recognized

    return np.select(conditions, choices, default=np.nan)

# Apply the function to create new columns for each F-value

def F1(epsilon, delta, Höhenwinkel):
    F1 = map_epsilon_to_column(epsilon,'F11') + map_epsilon_to_column(epsilon,'F12')*delta + map_epsilon_to_column(epsilon, 'F13')*(90-Höhenwinkel)

    return F1

def F2(epsilon, delta, Höhenwinkel):
    F2 = map_epsilon_to_column(epsilon,'F21') + map_epsilon_to_column(epsilon,'F22')*delta + map_epsilon_to_column(epsilon, 'F33')*(90-Höhenwinkel)

    return F2

#epsilon = ((dfStrahlung["DiffusHoriz"]+dfStrahlung["DirectHoriz"]*np.degrees(np.arcsin(dfSonnenstand["Höhenwinkel"])))/dfStrahlung["DiffusHoriz"]+kappa*(90-dfSonnenstand["Höhenwinkel"])**3)/(1+kappa*(90-dfSonnenstand["Höhenwinkel"])**3)
epsilon = 1.5 #warum funktioniert die Formel nicht?


dfPerez = pd.read_excel("Parametertabelle_Perez.xlsx")
dfPerez = dfPerez.transpose()
print(dfPerez)
#print(dfPerez[(dfPerez["epsilon"] < epsilon)[0:5,0:1]])

                  0      1       2      3      4      5      6
Unnamed: 0  epsilon    F11     F12    F13    F21    F22    F23
1               1.0 -0.008   0.588 -0.062  -0.06  0.072 -0.022
2             1.065   0.13   0.683 -0.151 -0.019  0.066 -0.029
3              1.23   0.33  0.0487 -0.221  0.055 -0.064 -0.026
4               1.5  0.568   0.187 -0.295  0.109 -0.152 -0.014
5              1.95  0.873  -0.392 -0.362  0.226 -0.462  0.001
6               2.8  1.132  -1.237 -0.412  0.288 -0.823  0.056
7               4.5   1.06    -1.6 -0.359  0.264 -1.127  0.131
8               6.2  0.678  -0.327  -0.25  0.156 -1.377  0.251


In [24]:
dfSonnenstand.loc[dfSonnenstand["Höhenwinkel"] < 5] = 0
dfSonnenstand["hilfsspalte"] = np.cos(np.radians(dfSonnenstand["Moduleinfallswinkel"]))/np.sin(np.radians(dfSonnenstand["Höhenwinkel"]))
dfSonnenstand["hilfsspalte"].clip(lower=0) #ersetzt den max-Operator in der Formel
dfSonnenstand["EdirGen"] = dfStrahlung["DirectHoriz"]*np.cos(np.radians(dfSonnenstand["Moduleinfallswinkel"]))/np.sin(np.radians(dfSonnenstand["Höhenwinkel"])) # falsch, da max Operator fehlt
dfSonnenstand = dfSonnenstand.drop(["hilfsspalte"], axis=1)
dfSonnenstand = dfSonnenstand.fillna(0)

dfSonnenstand["EreflGen"] = dfStrahlung["GlobalHoriz"]*A*(1/2)*(1-np.cos(np.radians(pv_hoehenwinkel)))




dfSonnenstand["hilfsspalte_a"] = np.cos(np.radians(dfSonnenstand["Moduleinfallswinkel"]))
dfSonnenstand["hilfsspalte_a"].clip(lower=0)
dfSonnenstand["hilfsspalte_b"] = np.sin(np.radians(dfSonnenstand["Höhenwinkel"]))
dfSonnenstand["hilfsspalte_b"].clip(lower=0.087)
dfSonnenstand["epsilon"] = ((dfStrahlung["DiffusHoriz"]+dfStrahlung["DirectHoriz"]*np.degrees(np.arcsin(np.radians(dfSonnenstand["Höhenwinkel"]))))/dfStrahlung["DiffusHoriz"]+kappa*(90-dfSonnenstand["Höhenwinkel"])**3)/(1+kappa*(90-dfSonnenstand["Höhenwinkel"])**3)
dfSonnenstand["delta"] = dfStrahlung['DiffusHoriz']/((1/np.sin(np.radians(dfSonnenstand['Höhenwinkel']))*E_0))
dfSonnenstand["EdiffGen"] = dfStrahlung["DiffusHoriz"]*(
    (1+np.cos(np.radians(pv_hoehenwinkel)))/2*(1-F1(dfSonnenstand['epsilon'],dfSonnenstand["delta"], dfSonnenstand["Höhenwinkel"])
    +(dfSonnenstand["hilfsspalte_a"]/dfSonnenstand["hilfsspalte_b"])*F1(dfSonnenstand['epsilon'],dfSonnenstand["delta"], dfSonnenstand["Höhenwinkel"]+
    F2(dfSonnenstand['epsilon'],dfSonnenstand["delta"], dfSonnenstand["Höhenwinkel"]*np.sin(np.radians(pv_hoehenwinkel))))))
dfSonnenstand = dfSonnenstand.drop(["hilfsspalte_a"], axis=1)
dfSonnenstand = dfSonnenstand.drop(["hilfsspalte_b"], axis=1)


dfSonnenstand["EgGen"] = dfSonnenstand["EdirGen"]+dfSonnenstand["EdiffGen"]+dfSonnenstand["EreflGen"]
dfEnergie = pd.DataFrame(dfSonnenstand['EgGen']*A*Modulwirkungsgrad*SonstigeVerluste)
dfSonnenstand[0:55]
#dfEnergie[20:50]

c:\Users\marku\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Azimut,Höhenwinkel,Moduleinfallswinkel,EdirGen,EreflGen,epsilon,delta,EdiffGen,EgGen
0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
2,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
3,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
4,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
5,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
6,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
7,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
8,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
9,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN


In [44]:
def Zeitgleichung(delta, rho, n, mez, laengengrad, breitengrad):
    """
    Berechnung der wahren Ortszeit über die Zeitgleichung.

    delta:
    rho: 
    n: Tagesnummer eines Jahres (1.1. n=1; 31.12. n=365)
    woz: Wahre Ortszeit [h]
    mez: Mitteleuropäische Zeit [h]
    längengrad [°]
    breitengrad [°]
    
    Args:
        delta: float
        rho: float
        n: int
        mez: float
        längengrad: float
        breitengrad: float


    Returns:
        woz: float

    """
    tau=360/365*n


    tau_rad = np.radians(tau)
    z = 0.008*np.cos(tau_rad)-0.122*np.sin(tau_rad)-0.052*np.cos(2*tau_rad) - 0.157*np.sin(2*tau_rad)-0.001*np.cos(3*tau_rad)-0.005*np.sin(3*tau_rad)

    woz = mez + z + 1/15*laengengrad-1
    
    return woz

def Moduleinfallswinkel(sonnenazimut, sonnenhöhe, modulazimut, modulneigungswinkel):
    """
    Args:
        


    Returns:
    
    
    """
    #falsch: 
    moduleinfallswinkel = np.arccos(-np.cos(np.radians(sonnenhöhe))*np.sin(np.radians(modulneigungswinkel)*np.cos(np.radians(sonnenazimut-modulazimut-180)+np.sin(np.radians(sonnenhöhe)*np.cos(modulneigungswinkel)))))
    return moduleinfallswinkel


In [45]:
dfStrahlung

,Year,Month,Day,Hour,DirectInclined,DiffuseInclined,Reflected,GlobalInclined,DirectHoriz,DiffusHoriz,GlobalHoriz,TopOfAtmosphere
0,2005,1,1,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2005,1,1,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2005,1,1,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2005,1,1,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2005,1,1,1.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2005,12,31,23.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35036,2005,12,31,23.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35037,2005,12,31,23.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35038,2005,12,31,23.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#3
def Strahlungsenergie(dfStrahlung,):
    
    EdiffGen = EdiffHor*max(0, np.cos())

In [47]:
"""
░░░░░░░█▐▓▓░████▄▄▄█▀▄▓▓▓▌█
░░░░░▄█▌▀▄▓▓▄▄▄▄▀▀▀▄▓▓▓▓▓▌█
░░░▄█▀▀▄▓█▓▓▓▓▓▓▓▓▓▓▓▓▀░▓▌█
░░█▀▄▓▓▓███▓▓▓███▓▓▓▄░░▄▓▐█▌
░█▌▓▓▓▀▀▓▓▓▓███▓▓▓▓▓▓▓▄▀▓▓▐█
▐█▐██▐░▄▓▓▓▓▓▀▄░▀▓▓▓▓▓▓▓▓▓▌█▌
█▌███▓▓▓▓▓▓▓▓▐░░▄▓▓███▓▓▓▄▀▐█
█▐█▓▀░░▀▓▓▓▓▓▓▓▓▓██████▓▓▓▓▐█
▌▓▄▌▀░▀░▐▀█▄▓▓██████████▓▓▓▌█▌
▌▓▓▓▄▄▀▀▓▓▓▀▓▓▓▓▓▓▓▓█▓█▓█▓▓▌█▌
█▐▓▓▓▓▓▓▄▄▄▓▓▓▓▓▓█▓█▓█▓█▓▓▓▐█
"""

'\n░░░░░░░█▐▓▓░████▄▄▄█▀▄▓▓▓▌█\n░░░░░▄█▌▀▄▓▓▄▄▄▄▀▀▀▄▓▓▓▓▓▌█\n░░░▄█▀▀▄▓█▓▓▓▓▓▓▓▓▓▓▓▓▀░▓▌█\n░░█▀▄▓▓▓███▓▓▓███▓▓▓▄░░▄▓▐█▌\n░█▌▓▓▓▀▀▓▓▓▓███▓▓▓▓▓▓▓▄▀▓▓▐█\n▐█▐██▐░▄▓▓▓▓▓▀▄░▀▓▓▓▓▓▓▓▓▓▌█▌\n█▌███▓▓▓▓▓▓▓▓▐░░▄▓▓███▓▓▓▄▀▐█\n█▐█▓▀░░▀▓▓▓▓▓▓▓▓▓██████▓▓▓▓▐█\n▌▓▄▌▀░▀░▐▀█▄▓▓██████████▓▓▓▌█▌\n▌▓▓▓▄▄▀▀▓▓▓▀▓▓▓▓▓▓▓▓█▓█▓█▓▓▌█▌\n█▐▓▓▓▓▓▓▄▄▄▓▓▓▓▓▓█▓█▓█▓█▓▓▓▐█\n'